In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", None)

In [4]:
trade = pd.read_csv("../data/trade-information.csv")
ir = pd.read_csv("../data/trade-price-ir-vegas.csv")
swap = pd.read_csv("../data/market-data-swap-rates.csv")
swaption = pd.read_csv("../data/market-data-swaption-vols.csv")

In [5]:
trade['bound width'] = trade['upper_bound'] - trade['lower_bound'] 

trade['month_pay_frequency'] = trade['pay_frequency'].str.removesuffix("M") # remove M from string
trade['month_pay_frequency'] = trade['month_pay_frequency'].astype(int) # convert to int


trade['year_maturity'] = trade['maturity'].str.removesuffix("Y") # remove Y from string
trade['year_maturity'] = trade['year_maturity'].astype(int)  # convert to int


In [6]:
trade

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397,0.0353,4,10


In [7]:
swap

,Date,Start Date,Tenor,Swap Rate
0,13/1/2021,13/1/2022,10y,1.081333
1,13/1/2021,13/1/2022,15y,1.265960
2,13/1/2021,13/1/2022,18M,0.221625
3,13/1/2021,13/1/2022,1y,0.195474
4,13/1/2021,13/1/2022,2y,0.279370
...,...,...,...,...
63175,12/1/2024,12/1/2039,2y,3.424505
63176,12/1/2024,12/1/2039,3y,3.393758
63177,12/1/2024,12/1/2039,4y,3.358897
63178,12/1/2024,12/1/2039,5y,3.319531


In [8]:
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687
...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706


In [9]:
# dictionary storing for tenor
dummy2YCMS = {'dummyTrade1': 2, 'dummyTrade2': 2, 'dummyTrade7': 2, 'dummyTrade8': 2}
dummy5YCMS = {'dummyTrade3': 5, 'dummyTrade4': 5, 'dummyTrade9': 5, 'dummyTrade10': 5}
dummy10YCMS = {'dummyTrade5': 10, 'dummyTrade6': 10, 'dummyTrade11': 10, 'dummyTrade12': 10}
YtoInt = {2:"2y", 5: "5y", 10: "10y"}

dummyTenorCMS = {**dummy2YCMS, **dummy5YCMS, **dummy10YCMS}

In [12]:
startfilter = swap['Date'].isin(ir['Value Date']) 
timefilter = swap['Tenor'].isin(YtoInt.values())
swap_rate = swap[startfilter & timefilter]

swap_rate = swap_rate.drop_duplicates(['Date', 'Tenor'], keep = 'first') # keep the earliest quoted start date of the swap rate
swap_rate

,Date,Start Date,Tenor,Swap Rate
34263,2/9/2022,4/9/2023,10y,2.737709
34267,2/9/2022,4/9/2023,2y,2.915516
34270,2/9/2022,4/9/2023,5y,2.724626
34506,6/9/2022,6/9/2023,10y,2.853063
34510,6/9/2022,6/9/2023,2y,3.026077
34513,6/9/2022,6/9/2023,5y,2.852453
34587,7/9/2022,7/9/2023,10y,2.781813
34591,7/9/2022,7/9/2023,2y,2.944991
34594,7/9/2022,7/9/2023,5y,2.770592
34668,8/9/2022,8/9/2023,10y,2.801851


In [13]:
ir['CMS Tenor'] = ir['Trade Name'].map(dummyTenorCMS).map(YtoInt)
zero = ir.loc[ir['Zero Rate Shock'] == 0]
# map the swap rate to the current zero rate shock 0 swap rate, fill zero rate change later


zero = zero.merge(swap_rate, left_on = ['Value Date', 'CMS Tenor'], right_on = ['Date', 'Tenor'], how = "left")


zero.drop(columns = ['Tenor', 'Date', 'Start Date'], inplace = True)
zero
## only map swap rate to zero shock data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate
0,2/9/2022,dummyTrade1,USD,0,-215402.6562,1y,4/9/2023,10y,5.016274,2y,2.915516
1,2/9/2022,dummyTrade10,USD,0,-616299.3692,1y,4/9/2023,10y,-59.820414,5y,2.724626
2,2/9/2022,dummyTrade11,USD,0,-608678.0110,1y,4/9/2023,10y,7.762595,10y,2.737709
3,2/9/2022,dummyTrade12,USD,0,-609360.6624,1y,4/9/2023,10y,25.329779,10y,2.737709
4,2/9/2022,dummyTrade2,USD,0,-215836.7872,1y,4/9/2023,10y,5.409384,2y,2.915516
...,...,...,...,...,...,...,...,...,...,...,...
87376,2/2/2023,dummyTrade2,USD,0,-217281.0660,15y,2/2/2038,10y,-0.003141,2y,2.846523
87377,2/2/2023,dummyTrade3,USD,0,-215534.8146,15y,2/2/2038,10y,0.027171,5y,2.674151
87378,2/2/2023,dummyTrade4,USD,0,-215949.0810,15y,2/2/2038,10y,0.022477,5y,2.674151
87379,2/2/2023,dummyTrade5,USD,0,-214200.3061,15y,2/2/2038,10y,-0.038096,10y,2.721939


In [14]:
ir = ir.merge(zero, on = ['Value Date', 'Trade Name', 'Zero Rate Shock', 'Trade Currency',  'TV',
       'Expiry Bucket', 'Expiry Date', 'Tenor Bucket', 'Vega', 'CMS Tenor'],  how = "left")
# merge zero shock swap rate data back to IR data

In [15]:
ir
# merged IR data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,NaN
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,NaN
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006,2y,NaN
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398,2y,NaN
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687,2y,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718,10y,NaN
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,NaN
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,NaN
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706,2y,NaN


In [14]:
#ir.to_csv("new ir.csv")

In [16]:
trade = trade.drop(columns = ["underlying", "pay_frequency", "maturity"])
trade.rename(columns={'trade name': 'Trade Name'}, inplace=True)
trade

,Trade Name,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,dummyTrade1,0.0042,0.0379,0.0337,6,5
1,dummyTrade2,0.0042,0.0379,0.0337,4,5
2,dummyTrade3,0.0044,0.0397,0.0353,6,5
3,dummyTrade4,0.0044,0.0397,0.0353,4,5
4,dummyTrade5,0.0046,0.0417,0.0371,4,5
5,dummyTrade6,0.0046,0.0417,0.0371,3,5
6,dummyTrade7,0.0042,0.0379,0.0337,6,10
7,dummyTrade8,0.0042,0.0379,0.0337,4,10
8,dummyTrade9,0.0044,0.0397,0.0353,6,10
9,dummyTrade10,0.0044,0.0397,0.0353,4,10


In [17]:
ir = ir.merge(trade, on = "Trade Name", how = "left")

In [18]:
ir

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity
0,2/9/2022,dummyTrade1,USD,-100,-227907.0988,1y,4/9/2023,10y,1.962246,2y,NaN,0.0042,0.0379,0.0337,6,5
1,2/9/2022,dummyTrade1,USD,-50,-222208.4010,1y,4/9/2023,10y,-3.812341,2y,NaN,0.0042,0.0379,0.0337,6,5
2,2/9/2022,dummyTrade1,USD,-25,-218960.9280,1y,4/9/2023,10y,4.471006,2y,NaN,0.0042,0.0379,0.0337,6,5
3,2/9/2022,dummyTrade1,USD,-10,-216872.4301,1y,4/9/2023,10y,4.333398,2y,NaN,0.0042,0.0379,0.0337,6,5
4,2/9/2022,dummyTrade1,USD,-5,-216146.3103,1y,4/9/2023,10y,5.679687,2y,NaN,0.0042,0.0379,0.0337,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2/2/2023,dummyTrade6,USD,75,-197572.2089,15y,2/2/2038,10y,0.104718,10y,NaN,0.0046,0.0417,0.0371,3,5
1048571,2/2/2023,dummyTrade6,USD,100,-186933.5679,15y,2/2/2038,10y,0.238781,10y,NaN,0.0046,0.0417,0.0371,3,5
1048572,2/2/2023,dummyTrade7,USD,-100,-680981.2765,15y,2/2/2038,10y,2.552405,2y,NaN,0.0042,0.0379,0.0337,6,10
1048573,2/2/2023,dummyTrade7,USD,-50,-656248.2494,15y,2/2/2038,10y,2.687706,2y,NaN,0.0042,0.0379,0.0337,6,10


In [19]:
ir.to_csv("complete ir.csv")